CTGAN artifacts? Chunks?
==

In TPS202111, kagglers discovered that labels had been flipped at random. As part of that, I (and others) ended up spending a lot of time checking out chunks and streaks of the data in order to try to see if there was a pattern to how the label flips had been chosen. We didn't find any, but we discovered a lot of weirdnesses about the data.

For the past few days, something hasn't been sitting right with me when I've been working on this TPS. I've been feeling like the cross validation folds almost make a bigger difference than model changes at this point on public LB. And the models I have that do best on local CV are no longer best on public LB.

So that got me thinking, is there some part of the training data that is somehow closer to the test data, that causes this to happen? And I decided to dig out some tools from the old TPS -- namely, I wanted to check if the data generation / distribution is, as it was back in TPS202111, dependent on ID somehow.

Here's the train class balance by ID -- this plot is based on cumulative counting of each class, we subtract the number of classes a "dice" with the total distribution set would've seen. Eg. when the `Cover_Type` line is at around 50 000 below, it means that, the data up until `Id=2.5e6` contains around 50 000 too many instances of that `Cover_Type` compared to the eventual result, which means that for IDs greater than `2.5e6`, we will expect the rate of `Cover_Type=2` to significantly decrease.

In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
import lightgbm

random.seed(64)
np.random.seed(64)

sns.set(style='darkgrid', context='notebook', rc={'figure.figsize': (16, 12), 'figure.frameon': False, 'legend.frameon': False})
df = pd.read_parquet('../input/tpsdec2021parquet/train.pq').set_index('Id')

label_summary = pd.DataFrame({
    f'Cover_Type_{cls}': (df.Cover_Type == cls).cumsum() - np.arange(len(df)) * (df.Cover_Type == cls).mean() 
    for cls in df.Cover_Type.unique()}
)
label_summary.plot.line(title='train class balance by id');

I thought it was fairly interesting that there's such a strong variation here. If you do folds by id without shuffling/stratifying, it's almost certainly the case that some of the folds have a distribution *much more* like test, than others.

Another thing I thought might be interesting to study, would be to attempt to see if this sort of thing also happens in the test set. We don't have true labels for that, so I've used the predictions from [this notebook](https://www.kaggle.com/remekkinas/tps-12-nn-tpu-pseudolabeling-0-95690) to compare, since we know that it's pretty close to the real labels.

Let's first have a look at test alone:

In [ ]:
df_test = pd.read_csv('../input/tps-12-nn-tpu-pseudolabeling-0-95690/tps12-pseudeo-submission.csv').set_index('Id')
label_summary_test = pd.DataFrame({
    f'Cover_Type_{cls}': (df_test.Cover_Type == cls).cumsum() - np.arange(len(df_test)) * (df_test.Cover_Type == cls).mean() 
    for cls in df_test.Cover_Type.unique()}
)
label_summary_test.plot.line(title='test class balance by id');

Then, we'll connect train and test:

In [ ]:
df = pd.concat([df[['Cover_Type']], df_test])

label_summary = pd.DataFrame({
    f'Cover_Type_{cls}': (df.Cover_Type == cls).cumsum() - np.arange(len(df)) * (df.Cover_Type == cls).mean() 
    for cls in df.Cover_Type.unique()}
)
label_summary.plot.line(title='class balance by id');

Well, would you look at that? To me, there's no abrubt change in the frequency of generated labels on the border between train and test.

Can we make use of that information? Is there something special that takes place on the inflection points of these curves? Does the same sort of thing happen in feature space? I don't really know. Either way, I thought these line charts were way too pretty to not show them to anybody. I'd be tempted to say from my finding that the dataset hasn't been shuffled after it was generated?

In [ ]:
df.groupby([(df.index // 20000) * 20000, df.Cover_Type]).size().unstack().fillna(0).plot.line(title='Cover_Type count per 20k Id');

Here's the same kind of cumsum-plot thinking for the bool/categorical features:

In [ ]:
del df


df = pd.read_parquet('../input/tpsdec2021parquet/train.pq').set_index('Id')
bool_cols = df.columns[df.dtypes == np.bool_]
delta_to_uniform = df[bool_cols].cumsum() - df[bool_cols].mean().to_numpy() * np.arange(len(df)).reshape(-1, 1)
delta_to_uniform[::20].plot.line()
plt.legend(loc='upper left', ncol=3);

It's easy to see that the incidence rate of these change throughout the dataset.

Comparison with Forest Cover Type Dataset
==

We know that the data in this TPS was generated from the forest cover type data set, so let's see if the original dataset has these artifacts:

In [ ]:
df = pd.read_csv('../input/forest-cover-type-dataset/covtype.csv', dtype=df.dtypes.to_dict()).assign(
    Id=lambda df: np.arange(len(df))
).set_index('Id')

label_summary = pd.DataFrame({
    f'Cover_Type_{cls}': (df.Cover_Type == cls).cumsum() - np.arange(len(df)) * (df.Cover_Type == cls).mean() 
    for cls in df.Cover_Type.unique()}
)
label_summary.plot.line(title='Original class balance by id');

In [ ]:
bool_cols = df.columns[df.dtypes == np.bool_]
delta_to_uniform = df[bool_cols].cumsum() - df[bool_cols].mean().to_numpy() * np.arange(len(df)).reshape(-1, 1)
delta_to_uniform[::2].plot.line()
plt.legend(loc='upper left', ncol=3);

I would say this actually has the same kind of behaviour as the synthetic dataset. As a control, let's verify that if we shuffle the data, this kind of thing doesn't happen:

In [ ]:
df = df.sample(frac=1)

label_summary = pd.DataFrame({
    f'Cover_Type_{cls}': (df.Cover_Type == cls).cumsum() - np.arange(len(df)) * (df.Cover_Type == cls).mean() 
    for cls in df.Cover_Type.unique()}
).set_index(pd.Series(np.arange(len(df)), name='Id'))
label_summary.plot.line(title='Original class balance by id');

Right, looking a lot more like what we'd expect, at imbalances of only a few hundred. I guess that this implies that both the synthetic dataset and the original are ordered by some attribute that has some sort of interaction with the label?

Is the data ordered?
==

In [this post](https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/292381), we can see that a classifier can separate data between the train and test data sets. At this point, I have a hunch that the data is ordered by something (other than `Id`), and that whatever it is, causes the features to somehow be ordered too.

Just for fun, let's check if it is predictable whether a sample has `Id` in the upper half of the training data:

In [ ]:
df = pd.read_parquet('../input/tpsdec2021parquet/train.pq')
y = df.Id > df.Id.median()
X = df.drop(columns=['Id', 'Cover_Type'])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=64)

for train_idx, val_idx in cv.split(X, y):
    train_set = lightgbm.Dataset(X.iloc[train_idx], label=y[train_idx])
    val_set = lightgbm.Dataset(X.iloc[val_idx], label=y[val_idx])
    booster = lightgbm.train(
        params={'objective': 'binary', 'metric': ['auc'], 'lambda': 2},
        train_set=train_set,
        valid_sets=[val_set],
        num_boost_round=400,
        early_stopping_rounds=10,
        verbose_eval=50,
    )

Right, there's a fairly strong indication that the data is ordered in some sort of way. How strongly ordered is it? Can we predict the index?

In [ ]:
for train_idx, val_idx in cv.split(X, y):
    train_set = lightgbm.Dataset(X.iloc[train_idx], label=train_idx)
    val_set = lightgbm.Dataset(X.iloc[val_idx], label=val_idx)
    booster = lightgbm.train(
        params={'objective': 'regression', 'metric': ['mae'], 'lambda': 2},
        train_set=train_set,
        valid_sets=[val_set],
        num_boost_round=300,
        early_stopping_rounds=10,
        verbose_eval=50,
    )

Don't rightly know what to make of this. No single feature column is sorted, not anywhere near sorted. Let's recheck that.

What does it mean to be sorted? Well, the next value is smaller than the current value, for each column, for all the rows. Or it could be that the next value is greater than the current value, for each column, for all the rows. Here's a quick check:

In [ ]:
(X <= X.shift()).mean()

In [ ]:
(X >= X.shift()).mean()

None of the columns are actually sorted. The Soil_Type columns that appear sorted just don't have any values. The other Soil Type columns are simply very sparse.

In [ ]:
pred_diff = val_idx - booster.predict(
    X.iloc[val_idx]
)

pd.Series(pred_diff).describe()

In [ ]:
sns.histplot(x=pred_diff, bins=50, kde=True);

In [ ]:
pd.DataFrame({
    'name': booster.feature_name(),
    'value': booster.feature_importance()
}).plot.barh(x='name', y='value');